# 7BR CMAP enrichment

2/8/2021


In [2]:
library(tidyverse)
# library(readr)
library(ggplot2)
library(clusterProfiler)
library(org.Hs.eg.db)
library(pheatmap)
library(ReactomePA)
library(annotate)
library(seqinr)

In [78]:
save_path = 'cmap_egenes/enrichment/'

In [140]:
path_genes = 'D_mpraanalyze_barcode_allelic//egene_gtex/'
diseases = sort(c('schizo','depress','bipolar','anxiety','attent','personality','panic','traum','autism','ocd'))
diseases_all = c(diseases, 'all')
get_genes = function(dz){
    file = str_c(path_genes,dz,'.txt')
    genes = read.table(file,header=F,stringsAsFactor=F)$V1
    print(dz)
    print(length(genes))
    return(genes)
}

get_entrez = function(genes){
    entrez_ids = bitr(genes, fromType="SYMBOL", toType="ENTREZID", OrgDb="org.Hs.eg.db")$ENTREZID
#     print(length(entrez_ids))
    return(entrez_ids)
#     return(bitr(genes, fromType="SYMBOL", toType="ENTREZID", OrgDb="org.Hs.eg.db"))
}

enrichr_res = function(entrez_genes, c_gmt){
    enrichr_res <- enricher(entrez_genes, TERM2GENE=c_gmt,                                            
                                            minGSSize=0,
                                            maxGSSize=1000,                                                
                                            pAdjustMethod = "fdr",
                                            qvalueCutoff = 1,
                                            pvalueCutoff = 1)
    
    if (!is.null(enrichr_res)){
        enrichr_res <- setReadable(enrichr_res, OrgDb=org.Hs.eg.db, keyType="ENTREZID")
        return(data.frame(enrichr_res))}
    return(data.frame())
}

filt_res = function(df){
    return(filter(df,p.adjust<0.05))
}

get_lengths = function(list_dfs){
    for (name in names(list_dfs)){
        print(name)
        if (is.data.frame(list_dfs[[name]])){
            print(dim(list_dfs[[name]])[1])
        }
        else{
            print(length(list_dfs[[name]]))
        }
    }
}

save_dfs = function(list_dfs, save_prefix){
    for (name in names(list_dfs)){
        if (dim(list_dfs[[name]])[1]>0){
            save_filepath = paste0(save_prefix, '_', name,'.csv')
            write.csv(list_dfs[[name]], save_filepath)            
        }

    }
}

                            
# get list of unique genes in list of lists
get_genes_unique = function(list_dfs){
    genelist = sort(unique(do.call('c',lapply(do.call('rbind',list_dfs)$geneID, function(x) strsplit(x,'/')[[1]]))))
    return(genelist)
}
run_enrichment = function(entrez_list, c_gmt, save_prefix){
    enrich_df_list =  sapply(entrez_list, function(x) enrichr_res(x,c_gmt))
    save_dfs(enrich_df_list, save_prefix)
    enrich_df_list_filt = sapply(enrich_df_list, filt_res)
    get_lengths(enrich_df_list_filt)
    print('getting unique genes --all')
    print(get_genes_unique (enrich_df_list)                        )
    print('getting unique genes --pval filt')
    print(get_genes_unique (enrich_df_list_filt)   )
    return(enrich_df_list_filt)
}
 

In [110]:
all_genes = get_genes('all')
length(all_genes)

[1] "all"
[1] 429


[1] 429

In [111]:
dz_gene_list_sym = sapply(diseases_all,get_genes)

[1] "anxiety"
[1] 18
[1] "attent"
[1] 46
[1] "autism"
[1] 1
[1] "bipolar"
[1] 129
[1] "depress"
[1] 55
[1] "ocd"
[1] 20
[1] "panic"
[1] 4
[1] "personality"
[1] 12
[1] "schizo"
[1] 308
[1] "traum"
[1] 5
[1] "all"
[1] 429


In [112]:
dz_gene_list_entrez = sapply(dz_gene_list_sym,get_entrez)
get_lengths(dz_gene_list_entrez)

'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Hs.eg.db"):
“4.35% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Hs.eg.db"):
“3.88% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Hs.eg.db"):
“5% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(genes, fromType = "SYMBOL", 

[1] "anxiety"
[1] 18
[1] "attent"
[1] 44
[1] "autism"
[1] 1
[1] "bipolar"
[1] 124
[1] "depress"
[1] 55
[1] "ocd"
[1] 19
[1] "panic"
[1] 4
[1] "personality"
[1] 12
[1] "schizo"
[1] 296
[1] "traum"
[1] 5
[1] "all"
[1] 415


# 1. enrichment of egenes drug target (from pt 1 of 7B)

In [141]:
save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target.gmt'
# save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target_cat.gmt'
c_repurpose <- read.gmt(save_repurpose_gmt)

dz_repurpose_drug_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_repurpose,
                          save_prefix=paste0(save_path, 'direct_drug_target'))


--> No gene can be mapped....

--> Expected input gene ID: 2548,1584,3356,2550,367,210

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 9970,367,1586,190,152,1019

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 1586,222962,210,50632,1814,1815

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 1812,51175,1814,3357,210,1803

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 1814,1021,3350,3356,1812,6506

--> return NULL...



[1] "anxiety"
[1] 3
[1] "attent"
[1] 0
[1] "autism"
[1] 0
[1] "bipolar"
[1] 19
[1] "depress"
[1] 8
[1] "ocd"
[1] 2
[1] "panic"
[1] 0
[1] "personality"
[1] 0
[1] "schizo"
[1] 2
[1] "traum"
[1] 0
[1] "all"
[1] 0
[1] "getting unique genes --all"
 [1] "ABCC8"   "AGER"    "AKT1"    "CHRNA2"  "CYP2D6"  "DDAH2"   "ERBB2"  
 [8] "FADS1"   "FGFR1"   "FGFR3"   "FYN"     "HAPLN4"  "ITPR3"   "NEU1"   
[15] "NFKB1"   "NT5C2"   "PTK2B"   "SLC12A4" "SV2A"    "TUBD1"  
[1] "getting unique genes --pval filt"
[1] "ABCC8"  "CYP2D6" "FADS1"  "FGFR1"  "FGFR3"  "HAPLN4" "ITPR3"  "SV2A"  
[9] "TUBD1" 


In [142]:
save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target_cat.gmt'
c_repurpose <- read.gmt(save_repurpose_gmt)

dz_repurpose_drug_cat_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_repurpose,
                          save_prefix=paste0(save_path, 'direct_drug_target_cat'))
dz_repurpose_drug_cat_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 2161,3718,4887,6256,54600,3361

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 3357,4986,27306,8644,2752,3356

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 5563,3770,1136,280,783,41

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 169522,3361,3417,11255,6755,1514

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 404037,1728,3620,3356,2618,554

--> return NULL...



[1] "anxiety"
[1] 0
[1] "attent"
[1] 0
[1] "autism"
[1] 0
[1] "bipolar"
[1] 0
[1] "depress"
[1] 0
[1] "ocd"
[1] 0
[1] "panic"
[1] 0
[1] "personality"
[1] 0
[1] "schizo"
[1] 0
[1] "traum"
[1] 0
[1] "all"
[1] 0
[1] "getting unique genes --all"
 [1] "ABCC8"   "AGER"    "AKT1"    "CHRNA2"  "CYP2D6"  "DDAH2"   "ERBB2"  
 [8] "FADS1"   "FGFR1"   "FGFR3"   "FYN"     "HAPLN4"  "ITPR3"   "NEU1"   
[15] "NFKB1"   "NT5C2"   "SLC12A4" "SV2A"    "TUBD1"  
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count


In [143]:
# dz_repurpose_drug_cat_filt

In [144]:
save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target_psych.gmt'
c_repurpose <- read.gmt(save_repurpose_gmt)

dz_repurpose_drug_psych_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_repurpose,
                          save_prefix=paste0(save_path, 'direct_drug_target_psych'))
dz_repurpose_drug_psych_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 3355,2564,1128,3350,3352,3358

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 3357,1806,147,3350,4967,1133

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 155,1131,1812,2554,1815,1133

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 155,3356,2568,153,160728,4057

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 4953,3355,1812,1132,2554,2557

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 147,2554,2557,3350,1814,4953

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 1812,6524,2555,1133,151,2568

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 9568,2562,3358,1133,151,153

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 2554,3269,3358,154,3355,3783

--> return NULL...



[1] "anxiety"
[1] 0
[1] "attent"
[1] 0
[1] "autism"
[1] 0
[1] "bipolar"
[1] 0
[1] "depress"
[1] 0
[1] "ocd"
[1] 0
[1] "panic"
[1] 0
[1] "personality"
[1] 0
[1] "schizo"
[1] 0
[1] "traum"
[1] 0
[1] "all"
[1] 0
[1] "getting unique genes --all"
[1] "NT5C2"
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<int>


In [145]:
save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target_psych_cat.gmt'
c_repurpose <- read.gmt(save_repurpose_gmt)

dz_repurpose_drug_psych_cat_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_repurpose,
                          save_prefix=paste0(save_path, 'direct_drug_target_psych_cat'))
dz_repurpose_drug_psych_cat_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 3360,3355,5226,801,4953,9568

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 1812,2554,1813,8911,122953,3269

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 1576,3362,2555,3351,3358,3315

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 151,2555,151,4067,2030,4057

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 1815,1131,153,147,4986,2554

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 4057,3784,1812,2568,93107,2558

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 2937,4967,5226,4985,22978,1812

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 771,57528,6283,1813,3783,2030

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 5226,3351,170850,3251,3251,1133

--> return NULL...



[1] "anxiety"
[1] 0
[1] "attent"
[1] 0
[1] "autism"
[1] 0
[1] "bipolar"
[1] 0
[1] "depress"
[1] 0
[1] "ocd"
[1] 0
[1] "panic"
[1] 0
[1] "personality"
[1] 0
[1] "schizo"
[1] 0
[1] "traum"
[1] 0
[1] "all"
[1] 0
[1] "getting unique genes --all"
[1] "NT5C2"
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<int>


In [146]:
save_cmap_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_up_crisp_drug.gmt'
c_cmap<- read.gmt(save_cmap_gmt)

dz_cmap_up_drug_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_cmap,
                          save_prefix=paste0(save_path, 'cmap_up_drug'))
dz_cmap_up_drug_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 27094,2700,1103,3683,55800,7915

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 151306,10060,146,1491,238,6286

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 4018,1543,5443,5688,1604,2739

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 9262,2739,6326,7010,3354,2700

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 7010,1577,6241,6330,6093,5724

--> return NULL...



[1] "anxiety"
[1] 0
[1] "attent"
[1] 0
[1] "autism"
[1] 0
[1] "bipolar"
[1] 0
[1] "depress"
[1] 0
[1] "ocd"
[1] 0
[1] "panic"
[1] 0
[1] "personality"
[1] 0
[1] "schizo"
[1] 0
[1] "traum"
[1] 0
[1] "all"
[1] 0
[1] "getting unique genes --all"
 [1] "ABCC8"   "AGER"    "AKT1"    "CHRNA2"  "CYP2D6"  "DDAH2"   "ERBB2"  
 [8] "FADS1"   "FGFR1"   "FGFR3"   "FYN"     "ITPR3"   "NEU1"    "NFKB1"  
[15] "NT5C2"   "PTK2B"   "SLC12A4" "SV2A"    "TUBD1"  
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count


In [147]:
save_cmap_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_dn_crisp_drug.gmt'
c_cmap<- read.gmt(save_cmap_gmt)

dz_cmap_dn_drug_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_cmap,
                          save_prefix=paste0(save_path, 'cmap_dn_drug'))
dz_cmap_dn_drug_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 6646,2147,29881,5327,84617,3567

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 2194,27,1131,3781,2051,4688

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 7226,2534,1786,7068,3291,83942

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 196,29943,1181,5027,5775,3291

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 2556,112714,134864,3156,29881,7422

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 5027,2556,5327,2560,4311,338442

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 5775,1586,285220,5565,23620,2158

--> return NULL...



[1] "anxiety"
[1] 0
[1] "attent"
[1] 0
[1] "autism"
[1] 0
[1] "bipolar"
[1] 0
[1] "depress"
[1] 0
[1] "ocd"
[1] 0
[1] "panic"
[1] 0
[1] "personality"
[1] 0
[1] "schizo"
[1] 0
[1] "traum"
[1] 0
[1] "all"
[1] 0
[1] "getting unique genes --all"
 [1] "AKT1"   "CHRNA2" "FGFR3"  "FYN"    "ITPR3"  "NFKB1"  "NT5C2"  "PTK2B" 
 [9] "SV2A"   "TUBD1" 
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>


In [148]:
save_cmap_gmt_cat = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_up_crisp_drug_cat.gmt'
c_cmap<- read.gmt(save_cmap_gmt_cat)

dz_cmap_up_drug_cat_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_cmap,
                          save_prefix=paste0(save_path, 'cmap_up_drug_cat'))
dz_cmap_up_drug_cat_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 3778,22926,2740,5167,9033,7010

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 1520,3354,2700,481,776,338567

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 152,5770,1586,1728,60598,468

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 6241,1080,210,1576,6332,4160

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 5153,177,1327,1394,6506,349149

--> return NULL...



[1] "anxiety"
[1] 0
[1] "attent"
[1] 0
[1] "autism"
[1] 0
[1] "bipolar"
[1] 0
[1] "depress"
[1] 0
[1] "ocd"
[1] 0
[1] "panic"
[1] 0
[1] "personality"
[1] 0
[1] "schizo"
[1] 0
[1] "traum"
[1] 0
[1] "all"
[1] 0
[1] "getting unique genes --all"
 [1] "ABCC8"   "AGER"    "AKT1"    "CHRNA2"  "CYP2D6"  "DDAH2"   "ERBB2"  
 [8] "FADS1"   "FGFR1"   "FGFR3"   "FYN"     "ITPR3"   "NEU1"    "NFKB1"  
[15] "NT5C2"   "PTK2B"   "SLC12A4" "SV2A"    "TUBD1"  
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count


In [149]:
save_cmap_gmt_cat = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_dn_crisp_drug_cat.gmt'
c_cmap<- read.gmt(save_cmap_gmt_cat)

dz_cmap_dn_drug_cat_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_cmap,
                          save_prefix=paste0(save_path, 'cmap_dn_drug_cat'))
dz_cmap_dn_drug_cat_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 1021,338557,9759,23743,8912,3784

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 27092,873,2194,5802,2048,2050

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 55244,3778,695,3718,9429,338442

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 5594,2534,3791,1513,5732,3791

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 113235,9311,5970,145226,1339,760

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 8654,5147,5424,51174,112714,4159

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 285220,1803,1141,55869,5423,747

--> return NULL...



[1] "anxiety"
[1] 0
[1] "attent"
[1] 0
[1] "autism"
[1] 0
[1] "bipolar"
[1] 0
[1] "depress"
[1] 0
[1] "ocd"
[1] 0
[1] "panic"
[1] 0
[1] "personality"
[1] 0
[1] "schizo"
[1] 0
[1] "traum"
[1] 0
[1] "all"
[1] 0
[1] "getting unique genes --all"
[1] "AKT1"   "CHRNA2" "FGFR3"  "FYN"    "ITPR3"  "NFKB1"  "PTK2B"  "SV2A"  
[9] "TUBD1" 
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>


In [150]:
#  enricher(dz_gene_list_entrez$schizo, TERM2GENE=c_repurpose,                                            
#                                             minGSSize=0,
#                                             maxGSSize=1000,                                                
#                                             pAdjustMethod = "none",
#                                             qvalueCutoff = 1,
#                                             pvalueCutoff = 0.9)

### interesting egenes (direct)

In [151]:
repurposing_info = read.csv('/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_20200324.txt',
                            stringsAsFactor=F,sep='\t',skip=9)
repurposing_info = repurposing_info%>%separate_rows(target,sep='[|]')
head(repurposing_info)

,pert_iname,clinical_phase,moa,target,disease_area,indication
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,ADRA2A,neurology/psychiatry,Parkinson's Disease
2,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,ADRA2B,neurology/psychiatry,Parkinson's Disease
3,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,ADRA2C,neurology/psychiatry,Parkinson's Disease
4,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,CALY,neurology/psychiatry,Parkinson's Disease
5,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,DRD1,neurology/psychiatry,Parkinson's Disease
6,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,DRD2,neurology/psychiatry,Parkinson's Disease


In [152]:
drug_egenes = get_genes_unique (dz_repurpose_drug)
length(drug_egenes)
drug_egenes_str = paste0(drug_egenes,collapse='|')
drug_egenes_info = data.frame(target=drug_egenes)%>%
    left_join(repurposing_info,by="target")
dim(drug_egenes_info)
length(unique(drug_egenes_info$target))

[1] 20

Warning message:
“Column `target` joining factor and character vector, coercing into character vector”


[1] 169   6

In [153]:
write.csv(drug_egenes_info, paste0(save_path,'drug_egenes_info.csv'))

In [157]:
drug_egenes_info%>% filter(target=='CHRNA2')

target,pert_iname,clinical_phase,moa,disease_area,indication
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
CHRNA2,acetylcholine,Launched,acetylcholine receptor agonist,neurology/psychiatry|gastroenterology|pulmonary,drowsiness|fatigue|headache|indigestion|chest congestion
CHRNA2,carbachol,Launched,cholinergic receptor agonist,ophthalmology,intraocular pressure
CHRNA2,choline,Phase 3,acetylcholine precursor,,
CHRNA2,cisatracurium,Launched,acetylcholine receptor antagonist,neurology/psychiatry|critical care,muscle relaxant|endotracheal intubation
CHRNA2,cytisine,Launched,acetylcholine receptor agonist,neurology/psychiatry,nicotinism
CHRNA2,decamethonium,Launched,acetylcholine receptor agonist,neurology/psychiatry,muscle relaxant
CHRNA2,dextromethorphan,Launched,glutamate receptor antagonist|sigma receptor agonist,pulmonary,cough suppressant
CHRNA2,galantamine,Launched,acetylcholinesterase inhibitor,neurology/psychiatry,senile dementia
CHRNA2,hexamethonium,Preclinical,cholinergic receptor antagonist,,
